In [2]:
import torch

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU detected:", torch.cuda.get_device_name(0))
else:
    print("⚠️ No GPU detected.")


CUDA available: True
GPU detected: NVIDIA GeForce RTX 4090 Laptop GPU


# Imports

In [3]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from torch.nn.functional import softmax
import torch.nn as nn


d:\new440\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Training

In [4]:
df = pd.read_csv("cgt-main/consolidated.csv", sep=";")

# check solidity exist, remove "nan" value
def has_source(fp_sol):
    sol_path = os.path.join(r"D:\new440\cgt-main\source", f"{fp_sol}.sol")
    return os.path.isfile(sol_path)

df = df[df["fp_sol"].apply(has_source)]

# transfer property_holds to label： "t" -> 1， "f" -> 0
df = df[df['property_holds'].isin(['t', 'f'])]
df['label'] = df['property_holds'].map({'t': 1, 'f': 0})

# read Solidity Source code, sol as txt
def read_source(fp_sol):
    sol_path = os.path.join(r"D:\new440\cgt-main\source", f"{fp_sol}.sol")
    try:
        with open(sol_path, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        print(f"file {sol_path} missing, continue")
        return ""

df['code'] = df['fp_sol'].apply(read_source)

# filter "nan" out，select code and label
data = df.dropna(subset=['code', 'label'])
data = data[data['code'] != ""]
solidity_data = []
for _, row in tqdm(data.iterrows(), total=len(data), desc="Loading Solidity files"):
    fp_sol = row["fp_sol"]
    sol_path = os.path.join("cgt-main", "source", f"{fp_sol}.sol")
    try:
        with open(sol_path, "r", encoding="utf-8") as f:
            code = f.read()
        solidity_data.append({
            "contract_name": row["contractname"],
            "code": code,
            "bug_type": row["property"],
            "swc_id": row["swc"],
            "dasp_id": row["dasp"]
        })
    except FileNotFoundError:
        pass
        
# JSON file saved
df_solidity = pd.DataFrame(solidity_data)
df_solidity.to_json("aaaaa.json", indent=4)

# train-validate separate
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['code'].tolist(), data['label'].tolist(), test_size=0.2, random_state=42
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions)
    }

# define PyTorch Dataset class
class SolidityDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        # auto cut code lenth
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Define weighted loss inside a custom Trainer
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Manually set class weights here
        weight = torch.tensor([1.0, 2.5]).to(logits.device)
        loss_fct = nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss


# load CodeBERT tokenizer and model form（microsoft/codebert-base）
model_name = "microsoft/codebert-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

# !!!!!!!!!!!!!!!!!!!!!!! need further explore
train_dataset = SolidityDataset(train_texts, train_labels, tokenizer)
val_dataset = SolidityDataset(val_texts, val_labels, tokenizer)

# Trainer define and tune
training_args = TrainingArguments(
    output_dir="./codebert-finetuned",  
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=15,
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# TRAINING
#trainer.train()
trainer.train(resume_from_checkpoint="./codebert-finetuned/checkpoint-4000")

predictions = trainer.predict(val_dataset)
logits = predictions.predictions
labels = predictions.label_ids

# Use custom threshold on softmax probs
probs = softmax(torch.tensor(logits), dim=1).numpy()
custom_preds = (probs[:, 1] > 0.4).astype(int)  # try 0.4 or lower

print(classification_report(labels, custom_preds, zero_division=0))

# Show result
eval_result = trainer.evaluate()
print("Evaluation results:", eval_result)

Loading Solidity files: 100%|██████████| 19456/19456 [00:01<00:00, 12257.01it/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\new440\.venv\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
4500,0.656800,0.643886,0.733299
5000,0.635800,0.651655,0.725591
5500,0.629800,0.656355,0.706835


              precision    recall  f1-score   support

           0       0.82      0.46      0.59      2825
           1       0.34      0.73      0.46      1067

    accuracy                           0.53      3892
   macro avg       0.58      0.59      0.52      3892
weighted avg       0.69      0.53      0.55      3892



Evaluation results: {'eval_loss': 0.6438862681388855, 'eval_accuracy': 0.7332990750256937, 'eval_runtime': 52.3619, 'eval_samples_per_second': 74.329, 'eval_steps_per_second': 9.301, 'epoch': 3.0}


# Testing

In [6]:
predictions = trainer.predict(val_dataset)
preds = predictions.predictions.argmax(axis=-1)
true_labels = predictions.label_ids

# Generate classification report
report = classification_report(true_labels, preds)
print(report)

              precision    recall  f1-score   support

           0       0.77      0.89      0.83      2825
           1       0.52      0.31      0.39      1067

    accuracy                           0.73      3892
   macro avg       0.65      0.60      0.61      3892
weighted avg       0.71      0.73      0.71      3892



In [7]:
trainer.save_model("best-codebert-model")

In [ ]:
import os
import glob
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.nn.functional import softmax
from tqdm import tqdm

def main():
    MODEL_DIR     = "best-codebert-model"               
    CONTRACTS_DIR = "smartbugs-wild-master/contracts"  
    OUTPUT_CSV    = "smartbugs_predictions.csv"        
    THRESHOLD     = 0.4                                 
    SAVE_INTERVAL = 2000                                 

    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
    model     = RobertaForSequenceClassification.from_pretrained(MODEL_DIR)
    model.to(device).eval()
    file_paths = glob.glob(os.path.join(CONTRACTS_DIR, "*.sol"))
    results = []


    for idx, path in enumerate(tqdm(file_paths, desc="Predicting contracts")):
        with open(path, "r", encoding="utf-8") as f:
            code = f.read()
        encoding = tokenizer(
            code,
            truncation=True,
            padding="max_length",
            max_length=512,
            return_tensors="pt"
        ).to(device)


        with torch.no_grad():
            outputs = model(**encoding)
        logits = outputs.logits
        probs  = softmax(logits, dim=-1).cpu().numpy()[0]

        pred_argmax = int(probs.argmax())
        pred_thresh = int(probs[1] > THRESHOLD)

        results.append({
            "file":          os.path.basename(path),
            "prob_no_bug":   float(probs[0]),
            "prob_bug":      float(probs[1]),
            "pred_argmax":   pred_argmax,
            "pred_thresh":   pred_thresh
        })

        if (idx + 1) % SAVE_INTERVAL == 0:
            pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
            print(f"Saved {idx+1} predictions so far to {OUTPUT_CSV}")

    pd.DataFrame(results).to_csv(OUTPUT_CSV, index=False)
    print(f"Saved all {len(results)} predictions to {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


Predicting contracts:   4%|▍         | 2008/47398 [00:44<14:13, 53.21it/s]

Saved 2000 predictions so far to smartbugs_predictions.csv


Predicting contracts:   8%|▊         | 4005/47398 [01:21<12:57, 55.82it/s]

Saved 4000 predictions so far to smartbugs_predictions.csv


Predicting contracts:  13%|█▎        | 6008/47398 [01:59<11:49, 58.37it/s]

...

Saved 44000 predictions so far to smartbugs_predictions.csv


Predicting contracts:  97%|█████████▋| 46004/47398 [29:10<01:13, 18.92it/s]

Saved 46000 predictions so far to smartbugs_predictions.csv


Predicting contracts: 100%|██████████| 47398/47398 [30:09<00:00, 26.19it/s]


Saved all 47398 predictions to smartbugs_predictions.csv


In [3]:
import pandas as pd

df = pd.read_csv("smartbugs_predictions.csv")

total = len(df)

num_bug = int(df["pred_thresh"].sum())

print(f"Total contrast: {total}")
print(f"Prediced valnerable: {num_bug}")
print(f"Percentage: {num_bug/total:.2%}")


Total contrast: 47398
Prediced valnerable: 33679
Percentage: 71.06%
